# #3 Rolling Up, Looking Back

In [1]:
import polars as pl

In [11]:
transactions = pl.scan_csv('coffee_shop_sales.csv', try_parse_dates=True)

monthly_sales = (
    transactions
    .group_by([
        pl.col("store"),
        pl.col("date").dt.year().alias("year"),
        pl.col("date").dt.month().alias("month")
    ])
    .agg([pl.col("sales").sum().alias("monthly_sales")])
    .sort(["store", "year", "month"])
    .with_columns(
        (
            pl.col("monthly_sales")
            - pl.col("monthly_sales")
              .shift(1)
              .over("store")
        ).alias("mom_sales_diff")
    )
)

monthly_sales.collect()

store,year,month,monthly_sales,mom_sales_diff
str,i32,i8,f64,f64
"""Astoria""",2023,1,27313.66,null
"""Astoria""",2023,2,25105.34,-2208.32
"""Astoria""",2023,3,32835.43,7730.09
"""Astoria""",2023,4,39477.61,6642.18
"""Astoria""",2023,5,52428.76,12951.15
…,…,…,…,…
"""Lower Manhattan""",2023,2,25320.05,-1223.38
"""Lower Manhattan""",2023,3,32888.68,7568.63
"""Lower Manhattan""",2023,4,39159.33,6270.65


In [12]:
monthly_sales.filter(pl.col("store") == "Astoria").collect()

store,year,month,monthly_sales,mom_sales_diff
str,i32,i8,f64,f64
"""Astoria""",2023,1,27313.66,null
"""Astoria""",2023,2,25105.34,-2208.32
"""Astoria""",2023,3,32835.43,7730.09
"""Astoria""",2023,4,39477.61,6642.18
"""Astoria""",2023,5,52428.76,12951.15
"""Astoria""",2023,6,55083.11,2654.35
